In [5]:
import pandas as pd 
import networkx as nx
import os
import random
from random import sample

def read_graph2(g, v):
    file_name = f'standard networks dataset{datasets[int(g)]}'
    G = nx.Graph()
    if v=='Karate':
        G = nx.karate_club_graph()
    elif v=='Erdos Renyi':
        # nodes = int(input("enter number of nodes?"))
        # edges= int(input("enter number of edges?"))
        G = nx.gnm_random_graph(500, 1500)
    elif v=='Barabasi_albert_graph':
        # nodes = int(input("enter number of nodes?"))
        # edges= int(input("enter number of edges?"))
        # p = int(input("enter P value?"))
        G = nx.barabasi_albert_graph(500, 3)
    else:
        ext = os.path.splitext(file_name)[1]
        if ext=='.edges':
            G = nx.read_adjlist(file_name, create_using = nx.Graph(), nodetype = int)
        elif ext=='.gml':
            G = nx.read_gml(file_name)
        elif ext=='.mtx':
            G = None
            #matrix = scipy.io.mmread(file_name)
            #G = nx.from_scipy_sparse_matrix(matrix)
        elif ext=='.txt':
            file = open(file_name, 'r')
            lines=  file.readlines()
            G = nx.Graph()
            for line in lines:
                if " " in line:
                    N = line.split(" ")
                else:
                    N = line.split("\t")
                G.add_edge(N[0], N[1])
    return G

names = ['facebook 107']
datasets = ['/facebook/107.edges']
# names = [ 'dolphins',
#           'polbooks',
#           'word_adjacencies',
#           'arenas-email',
#              'Karate',
#              'Erdos Renyi',
# #              'USAir97',
#              'circuits s208',
#              'circuits s420',
#              'circuits s838',
#              'E. Coli',
#              'Barabasi_albert_graph',
#              'facebook 0',
#              'facebook 107',
#              'facebook 348',
#              'facebook 414',
#              'facebook 686',
#              'facebook 1684',
# #              'bio-celegans',
#              'bn-macaque-rhesus_brain_2',
#              'soc-tribes',
#              'fb-pages-food',
#              'bn-cat-mixed-species_brain_1',
# #              'ca-sandi_auths',
#              'soc-firm-hi-tech']

# datasets = [ '/dolphins/dolphins.gml',
#              '/polbooks/out2.txt',
#              '/word_adjacencies.gml/word_adjacencies.gml',
#              '/arenas-email/out2.txt',
#              'Karate',
#              'Erdos Renyi',
# #              '/USAir97/USAir97.mtx',
#              '/circuits/s208_st.txt',
#              '/circuits/s420_st.txt',
#              '/circuits/s838_st.txt',
#              '/E. Coli/E. Coli.txt',
#              'Barabasi_albert_graph',
#              '/facebook/0.edges',
#              '/facebook/107.edges',
#              '/facebook/348.edges',
#              '/facebook/414.edges',
#              '/facebook/686.edges',
#              '/facebook/1684.edges',
# #              '/bio-celegans/bio-celegans.mtx',
#              '/bn-macaque-rhesus_brain_2/bn-macaque-rhesus_brain_2.txt',
#              '/soc-tribes/soc-tribes.txt',
#              '/fb-pages-food/fb-pages-food.txt',
#              '/bn-cat-mixed-species_brain_1/bn-cat-mixed-species_brain_1.txt',
# #              '/ca-sandi_auths/ca-sandi_auths.mtx',
#              '/soc-firm-hi-tech/soc-firm-hi-tech.txt']

# read the networks
networks = []
for i,v in enumerate(names):
    network = {}
    network['name'] = v
    network['path'] = datasets[i]
    network['graph'] = read_graph2(i, v)
    networks.append(network)
networks



[{'name': 'facebook 107',
  'path': '/facebook/107.edges',
  'graph': <networkx.classes.graph.Graph at 0x7f49c1a7d9d0>}]

In [6]:
names

['facebook 107']

In [7]:
from networkx.algorithms.community import greedy_modularity_communities, girvan_newman
def properties(G):
    
    GCC = nx.transitivity(G)
    ACC = nx.average_clustering(G)
    d = nx.density(G)
    r = nx.degree_assortativity_coefficient(G)    
    lcg = sorted(nx.connected_components(G), key=len, reverse=True)
    LCG = G.subgraph(lcg[0])    
    ASP = nx.average_shortest_path_length(LCG)
    diam = nx.diameter(LCG)

    communities = greedy_modularity_communities(G)
    mod = nx.community.modularity(G, communities)
    eff = round(nx.global_efficiency(G), 12)
    return  GCC, ACC, d, r, ASP, diam, mod, eff

network_properties = [properties(g['graph']) for g in networks]

GCCs = [i for i, _,_,_,_,_,_,_ in network_properties]
ACCs = [i for _, i,_,_,_,_,_,_ in network_properties]
ds   = [i for _, _,i,_,_,_,_,_ in network_properties]
rs   = [i for _, _,_,i,_,_,_,_ in network_properties]
ASPs = [i for _, _,_,_,i,_,_,_ in network_properties]
diam = [i for _, _,_,_,_,i,_,_ in network_properties]
mod = [i for _, _,_,_,_,_,i,_ in network_properties]
eff  = [i for _, _,_,_,_,_,_,i in network_properties]

df2 = pd.DataFrame({'Networks': names, 
                    'GCC': GCCs, 
                    'ACC': ACCs,
                    'Density': ds, 
                    'r': rs, 
                    'ASP': ASPs,
                    'Diameter': diam, 
                    'M': mod, 
                    'e' : eff}) 
df2

,Networks,GCC,ACC,Density,r,ASP,Diameter,M,e
0,facebook 107,0.504509,0.526405,0.050086,0.431569,2.951676,9,0.458112,0.385056


In [11]:
for g in networks:
    print(len(g['graph']), len(g['graph'].edges()))

1034 26749
